<a href="https://colab.research.google.com/github/nicolli-decastro/portifolio/blob/main/Analyzing_Matched_Lightning_Strike_and_Fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importing Libraries and Files

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from scipy.stats import mannwhitneyu
from scipy.stats import bootstrap
from google.colab import files

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google Colab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google Colab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# file address
# file = '/content/drive/MyDrive/FNN/2022_with_fire_mrms_v7 (1).csv'
file = '/content/2022_with_fire_mrms_v7_MATCHED_LIGHTNING.csv'
df_matched_fires = pd.read_csv(file)
df_matched_fires[:5]
# print(df_matched_fires.columns.values)

,Unnamed: 0,idx,dttime_utc,ltg_lat,ltg_lon,polarity,ell_smajor,ell_sminor,ell_angle,striketype,...,precip24h-1d_ltg,precip24h_day_fire,precip24h_fire,precip24h+1h_fire,precip24h+2h_fire,precip24h-1d_fire,distance,time_difference,lag_day,type
0,0,22735983,2022-05-04 19:41:02.999083757,26.359110,-80.416652,-15985,163,73,121,G,...,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,1.269497,0 days 03:26:57.000916243,0,promptly-detected
1,1,22087195,2022-05-02 20:48:02.668427944,29.715181,-83.139749,-20308,123,55,122,G,...,0.499999,0.0,0.000000,0.000000,0.000000,0.899998,0.323690,1 days 02:50:57.331572056,1,promptly-detected
2,2,15877204,2022-04-04 20:31:53.931598902,26.581265,-81.532187,-1874,232,100,117,G,...,3.799992,0.0,0.000000,0.000000,0.000000,122.400009,1.486524,1 days 07:08:06.068401098,1,promptly-detected
3,3,41312114,2022-06-10 19:09:13.076931000,29.726376,-81.830460,-1893,314,131,118,G,...,0.100000,0.1,0.000000,5.499989,5.499989,0.000000,1.651014,0 days 00:14:46.923069,0,promptly-detected
4,4,70075096,2022-08-02 19:19:56.876224995,29.015644,-80.992914,-37706,116,52,120,G,...,0.000000,0.0,5.599989,5.799988,5.799988,0.000000,1.245368,0 days 01:18:03.123775005,0,promptly-detected


# 2. Differenciating Between Promptly-Detected and Holdover Fires

In [ ]:
# Columns to create boxplots from
columns_to_plot = ['precip24h-1d_ltg', 'precip24h_ltg', 'precip24h+1h_ltg', 'precip24h+2h_ltg', 'precip24h_day_ltg']

# Ordering Fire Classes
fire_classes = ['Class 1 (0-5 ac)', 'Class 2 (6-50 ac)', 'Class 3 (51-500 ac)', 'Class 4 (> 500 ac)']

# Melting dataframe
df_melted = df_matched_fires.melt(id_vars=['size_class', 'type'], value_vars=columns_to_plot,
                                  var_name='Measurement', value_name='Precipitation (mm)')

# Box plot with color based on fire class and facet by fire type
fig = px.box(df_melted, x='Measurement', y='Precipitation (mm)', color='size_class',
             facet_col='type',
             title="Precipitation Distributions Across Fire Classes and Fire Type",
             labels={"Measurement": ""},
             category_orders={"Measurement": columns_to_plot, "size_class": fire_classes})  # <-- Adding order for size_class

color_map = {
    'Class 1 (0-5 ac)': 'blue',
    'Class 2 (6-50 ac)': 'red',
    'Class 3 (51-500 ac)': 'green',
    'Class 4 (> 500 ac)': 'yellow'
}

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])) # Facet column with only type of fire name
fig.update_traces(marker=dict(line=dict(width=2)))
fig.update_layout(showlegend=True, colorway=list(color_map.values()))  # use custom colors

# Display the plot
fig.show()


## 2.2 Count of promptly-detected and holdover fires for each fire class

In [ ]:
# Group by 'size_class' and 'type' and count the number of occurrences for each combination
fire_counts = df_matched_fires.groupby(['size_class', 'type']).size().unstack().reset_index()

# Rename columns for clarity
fire_counts.columns.name = None  # Remove the top-level column name
fire_counts = fire_counts.rename(columns={"promptly-detected": "Promptly-Detected Fires", "holdover": "Holdover Fires"})

print(fire_counts)


            size_class  Holdover Fires  Promptly-Detected Fires
0     Class 1 (0-5 ac)            36.0                    204.0
1    Class 2 (6-50 ac)            12.0                    115.0
2  Class 3 (51-500 ac)             4.0                     27.0
3   Class 4 (> 500 ac)             NaN                     12.0


## 2.3 Median Precipitation by Fire Class and Fire Type

In [ ]:
# Median precipitation values for each Fire Class and Fire Type
group = ['size_class','type']
medians_type_fire = df_matched_fires.groupby(group)[columns_to_plot].median()
medians_type_fire

precip24h-1d_ltg  precip24h_ltg  \
size_class          type                                                 
Class 1 (0-5 ac)    holdover                   0.000000       5.450002   
                    promptly-detected          0.000000       0.699999   
Class 2 (6-50 ac)   holdover                   0.150000       0.399999   
                    promptly-detected          0.000000       0.799998   
Class 3 (51-500 ac) holdover                   2.549995      10.349992   
                    promptly-detected          0.000000       0.399999   
Class 4 (> 500 ac)  promptly-detected          0.000000       0.000000   

                                       precip24h+1h_ltg  precip24h+2h_ltg  \
size_class          type                                                    
Class 1 (0-5 ac)    holdover                  17.149991         16.099993   
                    promptly-detected          5.149990          7.150011   
Class 2 (6-50 ac)   holdover                  18.499988         20.699997   
                    promptly-detected          5.899988          8.100009   
Class 3 (51-500 ac) holdover                  15.650007         19.699999   
                    promptly-detected          1.599997          1.599997   
Class 4 (> 500 ac)  promptly-detected          0.549999          1.349997   

                                       precip24h_day_ltg  
size_class          type                                  
Class 1 (0-5 ac)    holdover                    1.299997  
                    promptly-detected           1.949996  
Class 2 (6-50 ac)   holdover                    0.000000  
                    promptly-detected           1.299997  
Class 3 (51-500 ac) holdover                    6.150000  
                    promptly-detected           0.299999  
Class 4 (> 500 ac)  promptly-detected           0.649999

In [ ]:
# Variance precipitation values for each Fire Class and Fire Type
group = ['size_class','type']
variance_type_fire = df_matched_fires.groupby(group)[columns_to_plot].var()
variance_type_fire

precip24h-1d_ltg  precip24h_ltg  \
size_class          type                                                 
Class 1 (0-5 ac)    holdover                  86.619343     109.473433   
                    promptly-detected         35.932705     111.124735   
Class 2 (6-50 ac)   holdover                   8.257200       2.862034   
                    promptly-detected         20.839566      99.870515   
Class 3 (51-500 ac) holdover                  35.482497      92.696742   
                    promptly-detected         48.943718      48.070189   
Class 4 (> 500 ac)  promptly-detected          0.480225       2.284233   

                                       precip24h+1h_ltg  precip24h+2h_ltg  \
size_class          type                                                    
Class 1 (0-5 ac)    holdover                 147.307468        147.592829   
                    promptly-detected        256.246438        343.349486   
Class 2 (6-50 ac)   holdover                 195.111527        193.586035   
                    promptly-detected        281.010492        367.778155   
Class 3 (51-500 ac) holdover                 104.242570         39.922509   
                    promptly-detected         97.249389         96.258192   
Class 4 (> 500 ac)  promptly-detected         50.866362         87.235217   

                                       precip24h_day_ltg  
size_class          type                                  
Class 1 (0-5 ac)    holdover                   78.427319  
                    promptly-detected         202.441737  
Class 2 (6-50 ac)   holdover                   21.189679  
                    promptly-detected         122.714742  
Class 3 (51-500 ac) holdover                  294.126706  
                    promptly-detected         200.561004  
Class 4 (> 500 ac)  promptly-detected           3.601501

## 2.4 Combined Burned Area

In [ ]:
df_matched_fires['f_area'].sum()

56287.869999999995

# 3. Lightning Strike by Hour and Fire Type


In [ ]:
# Split the data based on the 'type'
df_promptly_detected = df_matched_fires[df_matched_fires['type'] == 'promptly-detected']
df_holdover = df_matched_fires[df_matched_fires['type'] == 'holdover']

In [ ]:
all_hours = list(range(24))

# Function to process each type of dataframe
def create_pivot_table(df):
    df['hour'] = pd.to_datetime(df['dttime_utc']).dt.hour
    # Calculate the bin edges based on the range of the 'precip24h+1h_ltg' and create 5 bins
    min_precip = df['precip24h+1h_ltg'].min()
    max_precip = df['precip24h+1h_ltg'].max()
    bin_size = (max_precip - min_precip) / 5
    bin_edges = [min_precip + i*bin_size for i in range(6)]
    labels = [f"{round(bin_edges[i], 2)}-{round(bin_edges[i+1], 0)}" for i in range(5)]
    df['precip_bins'] = pd.cut(df['precip24h+1h_ltg'], bins=bin_edges, labels=labels, right=False)
    return df.groupby(['hour', 'precip_bins']).size().unstack(fill_value=0).reindex(all_hours).fillna(0).astype(int).transpose().iloc[::-1]

# Create pivot tables for each type
pivot_table_promptly_detected = create_pivot_table(df_promptly_detected)
pivot_table_holdover = create_pivot_table(df_holdover)

# Closest hundredth greater than the max value
max_value_from_both = max(pivot_table_promptly_detected.values.max(), pivot_table_holdover.values.max())
rounded_max_value = int(np.ceil(max_value_from_both / 100.0) * 100)

# Convert pivot table data to logarithmic scale
log_pivot_table_promptly_detected = np.log(pivot_table_promptly_detected + 1)
log_pivot_table_holdover = np.log(pivot_table_holdover + 1)
tick_values = list(range(0, rounded_max_value + 1, 10))
log_tick_values = [np.log(val + 1) for val in tick_values]

# Create a subplot with 2 vertical plots
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=('Promptly Detected Fires', 'Holdover Fires'), vertical_spacing=0.1)

# Function to create heatmap
def create_heatmap(pivot_table, log_pivot_table):
    return go.Heatmap(
        z=log_pivot_table.values,
        x=log_pivot_table.columns,
        y=log_pivot_table.index,
        customdata=pivot_table.values,
        colorscale="Blues",
        zmin=0,
        zmax=np.log(pivot_table.values.max() + 1),
        hovertemplate='%{customdata} strikes<br>Hour: %{x}<br>Precipitation: %{y}<extra></extra>',
        colorbar=dict(
            title='Number of Strikes',
            tickvals=log_tick_values,
            ticktext=tick_values,
            tickmode='array'
        )
    )

# Add heatmaps to the subplots
fig.add_trace(create_heatmap(pivot_table_promptly_detected, log_pivot_table_promptly_detected), row=1, col=1)
fig.add_trace(create_heatmap(pivot_table_holdover, log_pivot_table_holdover), row=2, col=1)

# Reverse y-axis order and adjust the x-axis ticks for both subplots
fig.update_layout(yaxis_autorange="reversed", yaxis2_autorange="reversed",
                  title="Number of Lightning Strikes by Hour of the Day and Precipitation Range for precip24h_day_ltg")
fig.update_xaxes(tickvals=list(range(24)), ticktext=[str(i) for i in range(24)])

# Increase the gap between cells for better visualization
fig.update_traces(xgap=1, ygap=1)

fig.show()


<ipython-input-10-6fb6198778fd>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-6fb6198778fd>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-6fb6198778fd>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-6fb619

In [ ]:
# Split the data based on the 'type'
df_promptly_detected = df_matched_fires[df_matched_fires['type'] == 'promptly-detected']
df_holdover = df_matched_fires[df_matched_fires['type'] == 'holdover']

def generate_plot(df, title):
    # Calculate the bin edges
    min_precip = df['precip24h+1h_ltg'].min()
    max_precip = df['precip24h+1h_ltg'].max()
    bin_size = (max_precip - min_precip) / 10
    bin_edges = [min_precip + i*bin_size for i in range(11)]
    labels = [f"{round(bin_edges[i], 0)}-{round(bin_edges[i+1], 0)}" for i in range(10)]

    # Adjust the bin edges and bin the data
    bin_edges[0] = bin_edges[0] - 0.001
    bin_edges[-1] = bin_edges[-1] + 0.001
    df['precip_bins'] = pd.cut(df['precip24h+1h_ltg'], bins=bin_edges, labels=labels, right=True)
    df['hour'] = pd.to_datetime(df['dttime_utc']).dt.hour

    # Generate pivot table
    pivot_table = df.groupby(['hour', 'precip_bins']).size().unstack(fill_value=0)
    pivot_table = pivot_table[sorted(pivot_table.columns, key=lambda x: float(x.split('-')[0]))]

    # Colors
    custom_color_scale = [
        '#D2E4F4', '#A6C9E2', '#79AED0', '#4D93BE', '#24679B',
        '#FAD2D3', '#F79A9D', '#F26268', '#EB2B35', '#D00000'
    ]
    colors = custom_color_scale

    # Plotting
    fig = go.Figure()

    for idx, column in enumerate(pivot_table.columns):
        fig.add_trace(
            go.Bar(x=pivot_table.index,
                   y=pivot_table[column],
                   name=column,
                   marker_color=colors[idx])
        )
    fig.update_layout(
        title=title,
        xaxis_title='Hour of the Day',
        yaxis_title='Number of Lightning Strikes',
        barmode='stack',
        height=600
    )
    fig.update_layout(bargap=0.1)
    fig.update_xaxes(tickvals=list(range(0, 24)))
    fig.update_traces(hovertemplate="Number: %{y}<br>Hour: %{x}<extra></extra>")

    return fig

# Plot
fig_promptly_detected = generate_plot(df_promptly_detected, 'Promptly Detected Fires')
fig_holdover = generate_plot(df_holdover, 'Holdover Fires')

# Display
fig_promptly_detected.show()
fig_holdover.show()

<ipython-input-11-ed682d548715>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-ed682d548715>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-ed682d548715>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-11-ed68

# 4. Performing Mann-Whitney U Test

In [ ]:
promptly_detected_arr = df_promptly_detected.to_numpy()
holdover_arr = df_holdover.to_numpy()
print(holdover_arr)

[[5 41226544 '2022-06-10 18:00:33.427017212' ...
  '3 days 23:35:26.572982788' 3 'holdover']
 [6 21780209 '2022-05-01 18:01:48.047634840' ...
  '3 days 02:08:11.952365160' 3 'holdover']
 [33 83664454 '2022-08-24 17:21:34.315320253' ...
  '3 days 08:50:25.684679747' 3 'holdover']
 ...
 [383 89244578 '2022-09-02 20:23:54.250015259' ...
  '4 days 06:40:05.749984741' 4 'holdover']
 [405 33661147 '2022-05-27 15:22:32.018308640' ...
  '3 days 06:55:27.981691360' 3 'holdover']
 [406 39682081 '2022-06-07 23:28:08.057644367' ...
  '3 days 19:48:51.942355633' 3 'holdover']]


In [ ]:
# List of classes and measurements
fire_classes = ["Class 1 (0-5 ac)", "Class 2 (6-50 ac)", "Class 3 (51-500 ac)"]
measurements = ["precip24h-1d_ltg", "precip24h_ltg", "precip24h+1h_ltg", "precip24h+2h_ltg", "precip24h_day_ltg"]

# Data to store the results
data = []

# Loop through classes and measurements
for f_class in fire_classes:
    for measure in measurements:
        holdover_sample = df_holdover[df_holdover['size_class'] == f_class][measure].dropna()
        promptly_detected_sample = df_promptly_detected[df_promptly_detected['size_class'] == f_class][measure].dropna()

        # Calculate Median Values
        promptly_detected_median = df_promptly_detected[df_promptly_detected['size_class'] == f_class][measure].median()
        holdover_median = df_holdover[df_holdover['size_class'] == f_class][measure].median()

        # Perform Mann-Whitney U test
        stat, p = mannwhitneyu(holdover_sample, promptly_detected_sample, alternative='two-sided')

        # Determine significance
        significance = "Significant" if p < 0.10 else "Not Significant"

        # Append to data
        data.append([f_class, measure, promptly_detected_median, holdover_median, p, significance])

# Convert to DataFrame for display
results_df = pd.DataFrame(data, columns=['Fire Class', 'Precipitation Measurement', 'Promptly Detected Median', 'Holdover Median', 'P-Value', 'Significance'])
print(results_df)

             Fire Class Precipitation Measurement  Promptly Detected Median  \
0      Class 1 (0-5 ac)          precip24h-1d_ltg                  0.000000   
1      Class 1 (0-5 ac)             precip24h_ltg                  0.699999   
2      Class 1 (0-5 ac)          precip24h+1h_ltg                  5.149990   
3      Class 1 (0-5 ac)          precip24h+2h_ltg                  7.150011   
4      Class 1 (0-5 ac)         precip24h_day_ltg                  1.949996   
5     Class 2 (6-50 ac)          precip24h-1d_ltg                  0.000000   
6     Class 2 (6-50 ac)             precip24h_ltg                  0.799998   
7     Class 2 (6-50 ac)          precip24h+1h_ltg                  5.899988   
8     Class 2 (6-50 ac)          precip24h+2h_ltg                  8.100009   
9     Class 2 (6-50 ac)         precip24h_day_ltg                  1.299997   
10  Class 3 (51-500 ac)          precip24h-1d_ltg                  0.000000   
11  Class 3 (51-500 ac)             precip24h_ltg   

# 5. Performing Bootstrap Method

In [ ]:
# Split the data based on the 'type'
df_promptly_detected = df_matched_fires[df_matched_fires['type'] == 'promptly-detected']
df_holdover = df_matched_fires[df_matched_fires['type'] == 'holdover']

In [ ]:
def median_diff(data1, data2):
    return np.median(data1) - np. median(data2)

In [ ]:
# List of classes and measurements
fire_classes = ["Class 1 (0-5 ac)", "Class 2 (6-50 ac)", "Class 3 (51-500 ac)"]
measurements = ["precip24h-1d_ltg", "precip24h_ltg", "precip24h+1h_ltg", "precip24h+2h_ltg", "precip24h_day_ltg"]

data_bootstrap =[]

# Loop through fire classes and measurements
for f_class in fire_classes:
    for measure in measurements:

        # Filter the data for the specific fire class and measurement
        data_holdover = df_holdover[df_holdover['size_class'] == f_class][measure].dropna()
        data_promptly = df_promptly_detected[df_promptly_detected['size_class'] == f_class][measure].dropna()

        # Calculate Median Values
        promptly_detected_median = df_promptly_detected[df_promptly_detected['size_class'] == f_class][measure].median()
        holdover_median = df_holdover[df_holdover['size_class'] == f_class][measure].median()

        # Perform the bootstrap for the median difference
        boot_results = bootstrap((data_holdover, data_promptly), method='percentile', statistic=median_diff, n_resamples=1000, confidence_level=0.9)

        # Significance
        significance = "Significant" if boot_results.confidence_interval.low > 0 or boot_results.confidence_interval.high < 0 else "Not Significant"

        # Add results to the dataframe
        data_bootstrap.append([f_class, measure, promptly_detected_median, holdover_median, boot_results.confidence_interval.low, boot_results.confidence_interval.high, significance])

results_bootstrap_df = pd.DataFrame(data_bootstrap, columns=["Fire Class", "Precipitation Measurement", "Median Promptly-Detected", "Median Holdover", "Lower Bound", "Upper Bound", "Significance"])
print(results_bootstrap_df)

             Fire Class Precipitation Measurement  Median Holdover  \
0      Class 1 (0-5 ac)          precip24h-1d_ltg         0.000000   
1      Class 1 (0-5 ac)             precip24h_ltg         0.699999   
2      Class 1 (0-5 ac)          precip24h+1h_ltg         5.149990   
3      Class 1 (0-5 ac)          precip24h+2h_ltg         7.150011   
4      Class 1 (0-5 ac)         precip24h_day_ltg         1.949996   
5     Class 2 (6-50 ac)          precip24h-1d_ltg         0.000000   
6     Class 2 (6-50 ac)             precip24h_ltg         0.799998   
7     Class 2 (6-50 ac)          precip24h+1h_ltg         5.899988   
8     Class 2 (6-50 ac)          precip24h+2h_ltg         8.100009   
9     Class 2 (6-50 ac)         precip24h_day_ltg         1.299997   
10  Class 3 (51-500 ac)          precip24h-1d_ltg         0.000000   
11  Class 3 (51-500 ac)             precip24h_ltg         0.399999   
12  Class 3 (51-500 ac)          precip24h+1h_ltg         1.599997   
13  Class 3 (51-500 